# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [13]:
import pandas as pd
import numpy as np

# Load the dataset
house_sales = pd.read_csv('house_sales.csv')

# Replace non-standard missing values with NaN
missing_values = ["-", "missing", "NA", "N/A", "null", "NULL", "", "--"]
house_sales.replace(missing_values, np.nan, inplace=True)


# Calculate the number of missing values in the 'city' column after replacing non-standard values
missing_city = house_sales['city'].isnull().sum()

missing_city


73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [14]:
import pandas as pd

# Load the dataset
house_sales = pd.read_csv('house_sales.csv')

# Step 1: Handle missing values and non-standard representations
# Replace '--' in 'city' with "Unknown"
house_sales['city'] = house_sales['city'].replace('--', 'Unknown')

# Step 2: Fix 'area' column by removing 'sq.m.' and convert it to numeric
house_sales['area'] = house_sales['area'].str.replace(' sq.m.', '').astype(float)

# Step 3: Replace missing values as per the task instructions
house_sales['city'].fillna("Unknown", inplace=True)
house_sales = house_sales[house_sales['sale_price'].notna()]  # Remove rows with missing 'sale_price'
house_sales['sale_date'] = pd.to_datetime(house_sales['sale_date'], errors='coerce')
house_sales['sale_date'].fillna(pd.to_datetime("2023-01-01"), inplace=True)
house_sales['months_listed'].fillna(round(house_sales['months_listed'].mean(), 1), inplace=True)
house_sales['bedrooms'].fillna(round(house_sales['bedrooms'].mean()), inplace=True)
house_sales['house_type'] = house_sales['house_type'].replace({
    'Det.': 'Detached',
    'Semi': 'Semi-detached',
    'Terr.': 'Terraced'
})
house_sales['house_type'].fillna(house_sales['house_type'].mode()[0], inplace=True)
house_sales['area'].fillna(round(house_sales['area'].mean(), 1), inplace=True)

# Step 4: Create the cleaned DataFrame
clean_data = house_sales

# Display the cleaned data
print(clean_data.head())

   house_id        city  sale_price  ... bedrooms     house_type   area
0   1217792  Silvertown       55943  ...        2  Semi-detached  107.8
1   1900913  Silvertown      384677  ...        5       Detached  498.8
2   1174927   Riverford      281707  ...        6       Detached  542.5
3   1773666  Silvertown      373251  ...        6       Detached  528.4
4   1258487  Silvertown      328885  ...        5       Detached  477.1

[5 rows x 8 columns]


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [15]:
# Group by number of bedrooms and calculate average price and variance
price_by_rooms = clean_data.groupby('bedrooms').agg(
    avg_price=('sale_price', 'mean'),
    var_price=('sale_price', 'var')
).reset_index()

# Round results to 1 decimal place
price_by_rooms = price_by_rooms.round(1)

price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the training data
train_data = pd.read_csv('train.csv')

# Prepare the feature matrix (X) and target vector (y)
X_train = train_data.drop(['sale_price', 'house_id'], axis=1)
y_train = train_data['sale_price']

# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns

# Create a preprocessing pipeline for numeric and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

# Create a pipeline for preprocessing and modeling
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Train the model using the pipeline
pipeline.fit(X_train, y_train)

# Load the validation data
validation_data = pd.read_csv('validation.csv')

# Prepare the feature matrix for validation
X_validation = validation_data.drop(['house_id'], axis=1)

# Make predictions
predictions = pipeline.predict(X_validation)

# Create the result DataFrame
base_result = pd.DataFrame({'house_id': validation_data['house_id'], 'price': predictions})

# Display the results
print(base_result.head())

   house_id          price
0   1331375  124348.959174
1   1630115  304579.195013
2   1645745  401897.831750
3   1336775  110784.704999
4   1888274  307959.938600


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [17]:
from sklearn.ensemble import RandomForestRegressor

# Create a pipeline for preprocessing and modeling with RandomForest
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Train the model using the pipeline
pipeline_rf.fit(X_train, y_train)

# Make predictions
predictions_rf = pipeline_rf.predict(X_validation)

# Create the result DataFrame
compare_result = pd.DataFrame({'house_id': validation_data['house_id'], 'price': predictions_rf})

# Display the results
print(compare_result.head())

   house_id      price
0   1331375   82244.95
1   1630115  306043.08
2   1645745  404130.71
3   1336775  106886.32
4   1888274  266739.33
